In [1]:
import mne
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Try to use interactive backend if available, otherwise inline

In [2]:
# --- 1. Setup Paths ---
# Current folder (assuming this notebook is in code/digitspan/)
scripts_folder = Path.cwd()
root_folder = scripts_folder.parent.parent
data_path = root_folder / 'data' / 'ds003838' / 'sub-032'
data_folder = data_path / 'ecg'

subject_id = 'sub-032'
task = 'memory'

print(f"📂 Data folder: {data_folder}")

📂 Data folder: /home/martin/RESEARCH/thesis/code/brain_heart_psv_sdg/data/ds003838/sub-032/ecg


In [3]:
# --- 2. Load Signal (.set) ---
set_filename = f"{subject_id}_task-{task}_ecg.set"
set_path = data_folder / set_filename

if not set_path.exists():
    raise FileNotFoundError(f"Could not find {set_path}")

print(f"⏳ Loading {set_filename}...")
# Load the binary file with MNE
raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
print(raw.info)

⏳ Loading sub-032_task-memory_ecg.set...


/home/martin/miniforge3/envs/brain-heart/lib/python3.11/site-packages/pymatreader/utils.py:179: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


<Info | 8 non-empty values
 bads: []
 ch_names: PPG, ECG
 chs: 2 EEG
 custom_ref_applied: False
 dig: 2 items (2 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 2
 projs: []
 sfreq: 1000.0 Hz
>


/tmp/ipykernel_164600/1203144154.py:10: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)


In [4]:
# --- 3. Load Events (.tsv) ---
tsv_filename = f"{subject_id}_task-{task}_events.tsv"
tsv_path = data_folder / tsv_filename

if tsv_path.exists():
    print(f"⏳ Loading events from {tsv_filename}...")
    events_df = pd.read_csv(tsv_path, sep='\t')
    print("Events found:")
    print(events_df.head())
    
    # Check for value or trial_type columns
    # Prefer 'value' as it often contains specific descriptions (e.g., 'eyes opened')
    if 'value' in events_df.columns:
        print(f"Event values: {events_df['value'].unique()}")
        descriptions = events_df['value'].values
    elif 'trial_type' in events_df.columns:
        print(f"Event types: {events_df['trial_type'].unique()}")
        descriptions = events_df['trial_type'].values
    else:
        print("⚠️ No 'value' or 'trial_type' column found in events.")
        descriptions = ['Event'] * len(events_df)
    
    # Create MNE Annotations
    # MNE Annotations need onset (seconds), duration (seconds), and description
    onsets = events_df['onset'].values
    durations = events_df['duration'].values
        
    annotations = mne.Annotations(onsets, durations, descriptions)
    raw.set_annotations(annotations)
    print("✅ Annotations added to raw data.")
else:
    print(f"⚠️ Events file not found: {tsv_filename}")

⏳ Loading events from sub-032_task-memory_events.tsv...
Events found:
     onset  duration  sample  \
0    0.000       NaN       0   
1  684.496       1.0  684496   
2  686.559       1.0  686559   
3  688.587       1.0  688587   
4  690.630       1.0  690630   

                                          trial_type  response_time  \
0                                             STATUS            NaN   
1  control 01/13: listen to digit 1 (first) in 13...            NaN   
2  control 02/13: listen to digit 2 in 13 digit s...            NaN   
3  control 03/13: listen to digit 3 in 13 digit s...            NaN   
4  control 04/13: listen to digit 4 in 13 digit s...            NaN   

   stim_file     value  
0        NaN  boundary  
1        NaN    500113  
2        NaN    500213  
3        NaN    500313  
4        NaN    500413  
Event values: ['boundary' '500113' '500213' '500313' '500413' '500513' '500613' '500713'
 '500813' '500913' '501013' '501113' '501213' '501313' '500105' '500205

In [5]:
ecg_picks = mne.pick_types(raw.info, eeg=True)
len(ecg_picks)

2

In [7]:
# Set ECG type and pick it
ecg_ch = "ECG"
raw.set_channel_types({ecg_ch: "ecg"})
ecg_picks = mne.pick_channels(raw.info["ch_names"], include=[ecg_ch])
print(ecg_picks, [raw.ch_names[i] for i in ecg_picks])

# Alternatively, pick by type after setting
ecg_picks = mne.pick_types(raw.info, ecg=True)
print(ecg_picks, [raw.ch_names[i] for i in ecg_picks])

[1] ['ECG']
[1] ['ECG']


In [8]:
# Invert polarity of the ECG channel to get positive R-peaks
raw.apply_function(lambda x: -x, picks=[ecg_ch])

# Verify by plotting the single ECG channel
raw.copy().pick_channels([ecg_ch]).plot(duration=60, scalings='auto')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using qt as 2D backend.


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


Channels marked as bad:
none


In [ ]:
raw.copy().pick_channels([ecg_ch]).plot(duration=60, scalings='auto')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Using qt as 2D backend.


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
QFont::fromString: Invalid description 'Noto Sans Mono,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,400,0,0,0,0,0,0,0,0,0,0,1'
ERROR:root:Cannot activate multiple GUI eventloops


: 

In [ ]:
# --- 4. Visualize ---
# Plot the raw data with annotations
print("📊 Plotting data... ")
# Using 'scalings' to ensure ECG is visible. 'auto' usually works well.
raw.plot(duration=60, n_channels=len(raw.ch_names), scalings='auto', title=f"ECG - {subject_id} - {task}", show=True)

📊 Plotting data... 


ERROR:root:Cannot activate multiple GUI eventloops


: 